# Deep Learning Trading Model (A100 Optimized)

This notebook implements a deep learning model for trading using:

- LSTM + CNN + Attention architecture
- Multi-symbol training with market regime awareness
- A100 GPU acceleration
- Technical indicators and market context
- Custom loss functions with numerical stability

In [ ]:
# Install required packages
!pip install tensorflow pandas numpy scikit-learn yfinance plotly ta

import tensorflow as tf
import numpy as np
import pandas as pd
import ta
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, BatchNormalization,
    Conv1D, LayerNormalization, Activation, Add, Concatenate
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau, Callback
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime
import pickle

# Enable mixed precision for A100
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

# Configure memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

print("\nGPU Configuration:")
print("Number of GPUs:", len(gpus))
print("Mixed Precision:", tf.keras.mixed_precision.global_policy())
print("TensorFlow version:", tf.__version__)

# Verify GPU is being used
print("\nGPU Available:", tf.test.is_built_with_cuda())
print("GPU Device:", tf.test.gpu_device_name())

In [ ]:
def calculate_indicators(df):
    """Calculate technical indicators using the ta library."""
    # Initialize indicators
    bb_indicator = ta.volatility.BollingerBands(df['Close'])
    
    # Moving averages
    df['sma_20'] = ta.trend.sma_indicator(df['Close'], window=20)
    df['sma_50'] = ta.trend.sma_indicator(df['Close'], window=50)
    df['sma_200'] = ta.trend.sma_indicator(df['Close'], window=200)
    
    df['ema_10'] = ta.trend.ema_indicator(df['Close'], window=10)
    df['ema_20'] = ta.trend.ema_indicator(df['Close'], window=20)
    df['ema_50'] = ta.trend.ema_indicator(df['Close'], window=50)
    
    # Momentum
    df['rsi'] = ta.momentum.rsi(df['Close'], window=14)
    df['macd'] = ta.trend.macd_diff(df['Close'])
    df['mom_1d'] = df['Close'].pct_change(1)
    df['mom_5d'] = df['Close'].pct_change(5)
    df['mom_10d'] = df['Close'].pct_change(10)
    df['mom_20d'] = df['Close'].pct_change(20)
    
    # Volatility
    df['atr'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'])
    df['atr_pct'] = df['atr'] / df['Close']
    df['bb_high'] = bb_indicator.bollinger_hband()
    df['bb_mid'] = bb_indicator.bollinger_mavg()
    df['bb_low'] = bb_indicator.bollinger_lband()
    df['bb_width'] = (df['bb_high'] - df['bb_low']) / df['bb_mid']
    
    # Volume
    df['volume_ma_20'] = ta.trend.sma_indicator(df['Volume'], window=20)
    df['volume_ma_50'] = ta.trend.sma_indicator(df['Volume'], window=50)
    df['volume_ratio'] = df['Volume'] / df['volume_ma_20']
    df['volume_trend'] = df['Volume'].pct_change(5)
    
    # Price patterns
    df['high_low_range'] = (df['High'] - df['Low']) / df['Close']
    df['close_to_high'] = (df['High'] - df['Close']) / df['Close']
    df['close_to_low'] = (df['Close'] - df['Low']) / df['Close']
    
    # Trend strength
    df['trend_strength'] = np.where(
        df['Close'] > df['sma_50'],
        (df['Close'] - df['sma_50']) / df['sma_50'],
        -(df['sma_50'] - df['Close']) / df['sma_50']
    )
    
    return df.dropna()

In [ ]:
def calculate_cross_asset_features(tech_data):
    """Calculate cross-asset relationships between tech stocks."""
    features = {}
    
    # Calculate returns for correlation
    returns = {}
    for symbol in tech_data:
        returns[symbol] = tech_data[symbol]['Close'].pct_change()
    
    # Rolling correlations and relative strength
    for symbol in tech_data:
        # Initialize features for this symbol
        features[symbol] = pd.DataFrame(index=tech_data[symbol].index)
        
        # Calculate correlations with other stocks
        for other in tech_data:
            if other != symbol:
                # Rolling correlation
                corr = returns[symbol].rolling(20).corr(returns[other])
                features[symbol][f'corr_{other}'] = corr
                
                # Relative strength
                rel_strength = (tech_data[symbol]['Close'] / 
                               tech_data[symbol]['Close'].shift(20)) / \
                              (tech_data[other]['Close'] / 
                               tech_data[other]['Close'].shift(20))
                features[symbol][f'rel_strength_{other}'] = rel_strength
                
                # Relative volume
                rel_volume = (tech_data[symbol]['Volume'] / 
                             tech_data[symbol]['volume_ma_20']) / \
                            (tech_data[other]['Volume'] / 
                             tech_data[other]['volume_ma_20'])
                features[symbol][f'rel_volume_{other}'] = rel_volume
        
        # Sector-wide features
        tech_returns = pd.DataFrame([returns[s] for s in tech_data]).T
        
        # Stock's return vs sector average
        sector_avg_return = tech_returns.mean(axis=1)
        features[symbol]['sector_relative_return'] = \
            returns[symbol] - sector_avg_return
        
        # Stock's momentum vs sector average
        stock_mom = tech_data[symbol]['mom_20d']
        sector_mom = pd.DataFrame([tech_data[s]['mom_20d'] 
                                  for s in tech_data]).T.mean(axis=1)
        features[symbol]['sector_relative_momentum'] = \
            stock_mom - sector_mom
        
        # Stock's volatility vs sector average
        stock_vol = tech_data[symbol]['atr_pct']
        sector_vol = pd.DataFrame([tech_data[s]['atr_pct'] 
                                  for s in tech_data]).T.mean(axis=1)
        features[symbol]['sector_relative_volatility'] = \
            stock_vol - sector_vol
        
        # Number of sector stocks above their SMAs
        sma_signals = pd.DataFrame([
            tech_data[s]['Close'] > tech_data[s]['sma_50']
            for s in tech_data
        ]).T
        features[symbol]['sector_sma_strength'] = \
            sma_signals.sum(axis=1) / len(tech_data)
    
    return features

In [ ]:
def calculate_market_regimes(data):
    """Calculate market regime features from ETF data."""
    # Get market context data
    spy_data = data['SPY'].copy()
    qqq_data = data['QQQ'].copy()
    uvxy_data = data['UVXY'].copy()
    sqqq_data = data['SQQQ'].copy()
    tlt_data = data['TLT'].copy()
    
    # Market trend regime
    spy_data['market_trend'] = spy_data['Close'].pct_change(20)
    spy_data['market_regime'] = pd.qcut(
        spy_data['market_trend'],
        q=5,
        labels=['strong_down', 'down', 'neutral', 'up', 'strong_up']
    )
    
    # Tech sector regime
    qqq_data['tech_trend'] = qqq_data['Close'].pct_change(20)
    qqq_data['tech_regime'] = pd.qcut(
        qqq_data['tech_trend'],
        q=5,
        labels=['tech_strong_down', 'tech_down', 'tech_neutral', 'tech_up', 'tech_strong_up']
    )
    
    # Volatility regime
    uvxy_data['volatility'] = uvxy_data['Close'].pct_change(5)
    uvxy_data['volatility_regime'] = pd.qcut(
        uvxy_data['volatility'],
        q=5,
        labels=['very_low', 'low', 'normal', 'high', 'very_high']
    )
    
    # Bear market signals
    sqqq_data['bear_signal'] = sqqq_data['Close'].pct_change(10)
    sqqq_data['bear_regime'] = pd.qcut(
        sqqq_data['bear_signal'],
        q=5,
        labels=['strong_bull', 'bull', 'neutral', 'bear', 'strong_bear']
    )
    
    # Interest rate regime
    tlt_data['rates_trend'] = tlt_data['Close'].pct_change(20)
    tlt_data['rates_regime'] = pd.qcut(
        tlt_data['rates_trend'],
        q=5,
        labels=['rates_up_strong', 'rates_up', 'rates_neutral', 'rates_down', 'rates_down_strong']
    )
    
    # Additional market context features
    context = {
        'market_regime': spy_data['market_regime'],
        'tech_regime': qqq_data['tech_regime'],
        'volatility_regime': uvxy_data['volatility_regime'],
        'bear_regime': sqqq_data['bear_regime'],
        'rates_regime': tlt_data['rates_regime'],
        
        # Relative strength between markets
        'tech_vs_spy': (qqq_data['Close'] / qqq_data['Close'].shift(20)) / \
                      (spy_data['Close'] / spy_data['Close'].shift(20)),
                      
        # Volatility trends
        'vol_trend': uvxy_data['Close'].pct_change(5).rolling(5).mean(),
        
        # Market breadth
        'market_momentum': spy_data['mom_20d'],
        'tech_momentum': qqq_data['mom_20d'],
        
        # Risk measures
        'market_risk': spy_data['atr_pct'].rolling(10).mean(),
        'tech_risk': qqq_data['atr_pct'].rolling(10).mean()
    }
    
    return pd.DataFrame(context)

In [ ]:
def download_and_prepare_data(tech_stocks, market_etfs, start_date='2023-01-01'):
    """Download and prepare data for multiple symbols with cross-asset relationships."""
    all_data = {}
    all_symbols = tech_stocks + market_etfs
    
    # Download and process all data
    for symbol in all_symbols:
        print(f"Processing {symbol}...")
        ticker = yf.Ticker(symbol)
        df = ticker.history(start=start_date)
        df = calculate_indicators(df)
        all_data[symbol] = df
    
    # Calculate cross-asset features for tech stocks
    tech_data = {symbol: all_data[symbol] for symbol in tech_stocks}
    cross_features = calculate_cross_asset_features(tech_data)
    
    # Calculate market regimes and context
    market_context = calculate_market_regimes(all_data)
    
    # Combine all features
    processed_data = []
    for symbol in all_symbols:
        df = all_data[symbol].copy()
        
        # Add instrument type
        df['instrument_type'] = 'stock' if symbol in tech_stocks else 'etf'
        df['symbol'] = symbol
        
        # Add cross-asset features for tech stocks
        if symbol in tech_stocks:
            df = pd.concat([df, cross_features[symbol]], axis=1)
        
        # Add market context
        df = pd.concat([df, market_context], axis=1)
        
        processed_data.append(df)
    
    # Combine all data
    combined_data = pd.concat(processed_data, axis=0)
    
    return combined_data

# Define instruments
tech_stocks = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'AMD', 'META']
market_etfs = ['SPY', 'QQQ', 'UVXY', 'SQQQ', 'TLT']

# Download and prepare data
data = download_and_prepare_data(tech_stocks, market_etfs)

# Plot sample data
fig = go.Figure()
for symbol in tech_stocks:
    symbol_data = data[data['symbol'] == symbol]
    fig.add_trace(go.Scatter(
        x=symbol_data.index,
        y=symbol_data['Close'],
        name=symbol
    ))
fig.update_layout(title='Tech Stock Prices', xaxis_title='Date', yaxis_title='Price')
fig.show()